<a href="https://colab.research.google.com/github/Henzocosta/Alura/blob/main/Manipula%C3%A7%C3%A3o_de_dados_para_artigo_cba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manipulação da base de dados obrigatória para treinar os modelos de IA

In [ ]:
# Instalação das bibliotecas (pode não ser necessária no Colab, pois já vem pré-instalado)
# !pip install tensorflow pandas numpy

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
# Carregando os dados
data = pd.read_csv('dados_filtrados_z_100.csv') # Não esquecer de colocar o nome do arquivo que foi enviado no whats

data.dropna(inplace=True)  # Remove todas as linhas onde há pelo menos um valor nulo

In [ ]:
# Define a frequência como 10 para toda a coluna 'Frequência'
data['Frequência'] = 10


# Verifica se a transformação foi feita corretamente
data.head()

,Data,Hora,Frequência,z,speed,dir
0,2023-12-01,00:10:00,10,100,6.13,29.1
1,2023-12-01,00:20:00,10,100,6.04,27.1
2,2023-12-01,00:30:00,10,100,5.99,19.8
3,2023-12-01,00:40:00,10,100,5.39,22.5
4,2023-12-01,00:50:00,10,100,5.43,26.6


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3507 entries, 0 to 3840
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Data        3507 non-null   object 
 1   Hora        3507 non-null   object 
 2   Frequência  3507 non-null   int64  
 3   z           3507 non-null   int64  
 4   speed       3507 non-null   float64
 5   dir         3507 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 320.8+ KB


In [ ]:
# Combina as colunas 'Data' e 'Hora' em uma única coluna do tipo datetime
data['DataHora'] = pd.to_datetime(data['Data'] + ' ' + data['Hora'])

# Exclui as colunas 'Data' e 'Hora'
data.drop(columns=['Data', 'Hora'], inplace=True)

In [ ]:
data.head()

,Frequência,z,speed,dir,DataHora
0,10,100,6.13,29.1,2023-12-01 00:10:00
1,10,100,6.04,27.1,2023-12-01 00:20:00
2,10,100,5.99,19.8,2023-12-01 00:30:00
3,10,100,5.39,22.5,2023-12-01 00:40:00
4,10,100,5.43,26.6,2023-12-01 00:50:00


In [ ]:
# Converte a coluna 'DataHora' para a data juliana
data['DataJuliana'] = data['DataHora'].apply(lambda x: x.toordinal())

# Exclui as colunas 'DataHora'
data.drop(columns=['DataHora'], inplace=True)

# Visualizar as alterações feitas
data.head()

,Frequência,z,speed,dir,DataJuliana
0,10,100,6.13,29.1,738855
1,10,100,6.04,27.1,738855
2,10,100,5.99,19.8,738855
3,10,100,5.39,22.5,738855
4,10,100,5.43,26.6,738855
5,10,100,5.86,25.3,738855
6,10,100,6.05,20.7,738855
7,10,100,6.51,24.5,738855
8,10,100,6.63,23.5,738855
9,10,100,6.38,26.3,738855


In [ ]:
# Visualizar as informações da base de dados
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3507 entries, 0 to 3840
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Frequência   3507 non-null   int64  
 1   z            3507 non-null   int64  
 2   speed        3507 non-null   float64
 3   dir          3507 non-null   float64
 4   DataJuliana  3507 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 293.4 KB


# Treinamento do Modelo

In [ ]:
# Selecionando as colunas para o modelo
features = data[['Frequência','z', 'dir', 'DataJuliana']]
target = data['speed']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Construindo o modelo
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae'])

# Treinando o modelo
history = model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2)


Epoch 1/100
71/71 [==============================] - 2s 8ms/step - loss: 30.9201 - mae: 4.9423 - val_loss: 10.0562 - val_mae: 2.6028
Epoch 2/100
71/71 [==============================] - 0s 4ms/step - loss: 6.5856 - mae: 2.0623 - val_loss: 6.3184 - val_mae: 1.9857
Epoch 3/100
71/71 [==============================] - 0s 4ms/step - loss: 5.3516 - mae: 1.8532 - val_loss: 5.6987 - val_mae: 1.8716
Epoch 4/100
71/71 [==============================] - 0s 4ms/step - loss: 4.9336 - mae: 1.7728 - val_loss: 5.3445 - val_mae: 1.8065
Epoch 5/100
71/71 [==============================] - 0s 4ms/step - loss: 4.6786 - mae: 1.7180 - val_loss: 5.1449 - val_mae: 1.7683
Epoch 6/100
71/71 [==============================] - 0s 4ms/step - loss: 4.4979 - mae: 1.6862 - val_loss: 4.9908 - val_mae: 1.7465
Epoch 7/100
71/71 [==============================] - 0s 4ms/step - loss: 4.3777 - mae: 1.6666 - val_loss: 4.9202 - val_mae: 1.7316
Epoch 8/100
71/71 [==============================] - 0s 5ms/step - loss: 4.3203 -

In [ ]:
# Avaliação do modelo
model.evaluate(X_test_scaled, y_test)

22/22 [==============================] - 0s 2ms/step - loss: 3.9223 - mae: 1.5419


[3.9223411083221436, 1.5418599843978882]